In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_marix_car'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_marix_car


In [4]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score
from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [5]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engneering


In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue #if the df is a list then skip it

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat: #if the feature contains suffic __cat
    df[feat] = factorized_values #then assign the same value agian a__cat = a__cat
  else:
    df[feat + SUFFIX_CAT] = factorized_values #else add a suffix

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int( str(x).split('cm')[0].replace(' ', (''))) )
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]) )
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == "None" else int(x))

In [0]:
def run_model(model,feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error') #neg mean 
  return np.mean(scores), np.std(scores) #almost 2 times better result

In [10]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','param_wersja__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']


xgb_params = {
    'max_depth' : 5 ,
    'n_estimators' : 50,
    'learning_rate': 0.1,
    'seed': 0
}


run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:16:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:16:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:16:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

## Testing multiple parameters using hyperopt over 25 iterations


In [0]:
#obj_func text input from space described below, this function is our black box

def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK} 
  #return dictionary - status ok calculate the iteration with set of params sucessfully

#space - model will get 7 parameters, some are static, some are dynamic - we tell 
#in choice we have a list we iterate over
#quniform for floats
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',  np.arange(0.05, 0.31, 0.05) ), 
    'max_depth' :       hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)), #from 5 to 16 by 1
    'subsample' :       hp.quniform('subsample', 0.5, 1, 0.05), #range from 0.5 to 1 by 0.05 step
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective' :       'reg:squarederror',
    'n_estmimators':    100,
    'seed' :            0,

}


In [0]:
#run
best = fmin(obj_func, xgb_reg_params, algo = tpe.suggest, max_evals=25)
best

Training with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 11, 'n_estmimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.2, 'max_depth': 13, 'n_estmimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.25, 'max_depth': 8, 'n_estmimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.3, 'max_depth': 15, 'n_estmimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 14, 'n_estmimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample